## Imports

In [22]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad

## Load data

In [23]:
df = pd.read_csv(r'Dataset\data.csv', low_memory=False)
x = df['no2'].dropna().values

print(f"Dataset loaded successfully")
print(f"Total data points: {len(x)}")
print(f"NO2 range: [{x.min():.2f}, {x.max():.2f}]")

Dataset loaded successfully
Total data points: 419509
NO2 range: [0.00, 876.00]


## Transformation Parameters

In [24]:
r = 102303759

a_r = 0.05 * (r % 7)
b_r = 0.3 * (r % 5 + 1)

print(f"Roll number: {r}")
print(f"a_r = 0.05 × (r mod 7) = 0.05 × {r % 7} = {a_r}")
print(f"b_r = 0.3 × (r mod 5 + 1) = 0.3 × {r % 5 + 1} = {b_r}")

Roll number: 102303759
a_r = 0.05 × (r mod 7) = 0.05 × 5 = 0.25
b_r = 0.3 × (r mod 5 + 1) = 0.3 × 5 = 1.5


## Apply Transformations

In [25]:
z = x + a_r * np.sin(b_r * x)

print(f"\nTransformation: z = x + {a_r} * sin({b_r} * x)")
print(f"Transformed data statistics:")
print(f"  Mean: {np.mean(z):.4f}")
print(f"  Std: {np.std(z):.4f}")
print(f"  Range: [{z.min():.2f}, {z.max():.2f}]")


Transformation: z = x + 0.25 * sin(1.5 * x)
Transformed data statistics:
  Mean: 25.8181
  Std: 18.4950
  Range: [0.00, 876.18]


## Define MLE function

In [26]:
def negative_log_likelihood(params, data):
    lambda_param, mu = params
    
    if lambda_param <= 0:
        return np.inf
    
    n = len(data)
    c = np.sqrt(lambda_param / np.pi)
    
    log_likelihood = n * np.log(c) - lambda_param * np.sum((data - mu) ** 2)
    
    return -log_likelihood

## Estimate Parameters

In [27]:
initial_guess = [1.0 / (2 * np.var(z)), np.mean(z)]

result = minimize(
    negative_log_likelihood, 
    initial_guess, 
    args=(z,), 
    method='L-BFGS-B', 
    bounds=[(1e-10, None), (None, None)]
)

lambda_est = result.x[0]
mu_est = result.x[1]
c_est = np.sqrt(lambda_est / np.pi)

print("Maximum Likelihood Estimation Results:")
print(f"λ (lambda) = {lambda_est:.12f}")
print(f"μ (mu)     = {mu_est:.12f}")
print(f"c          = {c_est:.12f}")
print(f"\nOptimization converged: {result.success}")

Maximum Likelihood Estimation Results:
λ (lambda) = 0.001461705241
μ (mu)     = 25.818063543032
c          = 0.021570239426

Optimization converged: True


## Validate PDF

In [28]:
def pdf(z_val, lambda_param, mu, c):
    return c * np.exp(-lambda_param * (z_val - mu) ** 2)

z_min, z_max = z.min() - 50, z.max() + 50
integral, error = quad(pdf, z_min, z_max, args=(lambda_est, mu_est, c_est))

print(f"PDF Validation:")
print(f"∫ p(z) dz = {integral:.6f}")
print(f"Integration error: {error:.2e}")

if abs(integral - 1.0) < 0.01:
    print("PDF is valid (integrates to approximately 1)")

PDF Validation:
∫ p(z) dz = 0.999979
Integration error: 5.48e-09
PDF is valid (integrates to approximately 1)
